In [1]:
# SCHEDULED GENERATION AND AVAILABILITY

import pandas as pd
import numpy as np

from entsoe import EntsoePandasClient
from entsoe.mappings import Area, NEIGHBOURS, lookup_area
from entsoe.parsers import parse_generation
from typing import Union, Optional, Dict, List, Literal

client = EntsoePandasClient(api_key="b18dfce9-f1e3-4d07-822f-4abd1438e602")

In [8]:
start = pd.Timestamp('20220901', tz='Europe/Amsterdam')
end = pd.Timestamp('20220902', tz='Europe/Amsterdam')
#end = pd.Timestamp('20230331', tz='Europe/Amsterdam')

countries = ['AT', 'BE', 'CZ', 'DE_LU', 'FR', 'HR', 'HU', 'NL', 'PL', 'RO', 'SI', 'SK']

In [6]:
df_plants = pd.read_excel('./plants.xlsx', index_col=0)
df_gen = pd.read_excel('../cost_curves/data/generation.xlsx', index_col=0)

In [14]:
df_outages = pd.DataFrame()

for country_code in [x for x in countries if x not in ['HR', 'SI']]:
    print(country_code)
    df_outages_temp = client.query_unavailability_of_generation_units(country_code, start=start, end=end)
    df_outages_temp = df_outages_temp.tz_localize(None)
    df_outages_temp['end'] = df_outages_temp['end'].dt.tz_localize(None)
    df_outages_temp['start'] = df_outages_temp['start'].dt.tz_localize(None)
    df_outages = pd.concat([df_outages, df_outages_temp], ignore_index=True)

display(df_outages)

AT
BE
CZ
DE_LU
FR
HU
NL
PL
RO
SK


,avail_qty,biddingzone_domain,businesstype,curvetype,docstatus,end,mrid,nominal_power,plant_type,production_resource_id,production_resource_location,production_resource_name,pstn,qty_uom,resolution,revision,start
0,0,AT,Planned maintenance,A03,Cancelled,2023-01-01 00:00:00,be3Cxp6yr6iQNN07LZSgJA,140.0,Fossil Gas,14W-PROD-LAU---8,intra_zonal,KW Leopoldau,1,MAW,PT15M,2,2022-01-01 00:00:00
1,0,AT,Planned maintenance,A03,None,2023-01-01 00:00:00,KqPp9Q65jWB93v_xwyKxEA,140.0,Fossil Gas,14W-PROD-LAU---8,intra_zonal,KW Leopoldau,1,MAW,PT15M,1,2022-01-01 00:00:00
2,0,AT,Planned maintenance,A03,Cancelled,2022-10-01 00:00:00,izI9pAkJfrJWoG-IWvxnzQ,417.0,Fossil Gas,14W-WGM-KW-----J,intra_zonal,GDK-Mellach,1,MAW,PT15M,2,2022-04-01 00:00:00
3,0,AT,Planned maintenance,A03,Cancelled,2023-01-01 00:00:00,zEV-sw-wWzFH9CpblHCxNg,421.0,Fossil Gas,14W-WGM-KW-----J,intra_zonal,GDK-Mellach,1,MAW,PT15M,2,2022-01-01 00:00:00
4,215,AT,Planned maintenance,A03,Cancelled,2022-09-08 17:00:00,dl84Q0Z9OaV_DyJSWhyh5Q,430.0,Hydro Pumped Storage,14W-DRP-KW-----M,intra_zonal,Reisseck 2,1,MAW,PT15M,2,2022-08-29 07:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
598,0,SK,Planned maintenance,A03,None,2022-09-05 03:00:00,kafPXCjrjCRSV0K7jLipNQ,250.0,Nuclear,24WV--EBO------8,Jaslovské Bohunice,Bohunice,1,MAW,PT60M,118,2022-09-01 04:00:00
599,0,SK,Planned maintenance,A03,None,2022-10-12 00:00:00,mvdn96ggDQX7GxnyhwsqFg,110.0,Fossil Hard coal,24WV--EVO------D,Vojany,Vojany,1,MAW,PT60M,165,2022-08-01 00:00:00
600,0,SK,Planned maintenance,A03,None,2022-09-05 03:00:00,lmAiEJe0Oe4AwBWnDliJ_w,250.0,Nuclear,24WV--EBO------8,Jaslovské Bohunice,Bohunice,1,MAW,PT60M,118,2022-09-01 04:00:00
601,0,SK,Planned maintenance,A03,None,2022-10-01 00:00:00,MJ_OvK8qjXZz41maCvFw_A,110.0,Fossil Hard coal,24WV--EVO------D,Vojany,Vojany,1,MAW,PT60M,132,2022-08-20 00:00:00


In [20]:
df_generation = client.query_generation_per_plant('NL', start=start, end=end)
display(df_generation)

,Amer 9,Borssele 30,Merwedekanaal 12,Claus C,Delesto 2,Diemen 33,Diemen 34,EDH,Elsta 1,Eems 20,...,Pergen 2,Rijnmond REC-A,Rijnmond REC-B,Rijnmond REC-X,RoCa 3,Sloecentrale 20,Sloecentrale 10,Swentibold 1,Velsen 24,Velsen 25
,Fossil Hard coal,Nuclear,Fossil Gas,Fossil Gas,Fossil Gas,Fossil Gas,Fossil Gas,Fossil Gas,Fossil Gas,Fossil Gas,...,Fossil Gas,Fossil Gas,Fossil Gas,Fossil Gas,Fossil Gas,Fossil Gas,Fossil Gas,Fossil Gas,Fossil Gas,Fossil Gas
newlevel,Actual Aggregated,Actual Aggregated,Actual Aggregated,Actual Aggregated,Actual Aggregated,Actual Aggregated,Actual Aggregated,Actual Aggregated,Actual Aggregated,Actual Aggregated,...,Actual Aggregated,Actual Aggregated,Actual Aggregated,Actual Aggregated,Actual Aggregated,Actual Aggregated,Actual Aggregated,Actual Aggregated,Actual Aggregated,Actual Aggregated
2022-09-01 00:00:00+02:00,562.0,465.0,0.0,516.0,0.0,0.0,279.0,0.0,125.0,0.0,...,90.0,0.0,0.0,0.0,0.0,0.0,0.0,211.0,0.0,251.0
2022-09-01 01:00:00+02:00,562.0,465.0,0.0,436.0,0.0,0.0,286.0,0.0,125.0,0.0,...,90.0,0.0,0.0,0.0,0.0,0.0,0.0,164.0,0.0,218.0
2022-09-01 02:00:00+02:00,562.0,465.0,0.0,468.0,0.0,0.0,407.0,0.0,125.0,0.0,...,91.0,0.0,0.0,0.0,0.0,0.0,0.0,208.0,0.0,236.0
2022-09-01 03:00:00+02:00,561.0,465.0,0.0,671.0,0.0,0.0,421.0,0.0,125.0,0.0,...,90.0,0.0,0.0,0.0,0.0,0.0,0.0,222.0,0.0,230.0
2022-09-01 04:00:00+02:00,564.0,465.0,0.0,775.0,0.0,6.0,421.0,0.0,125.0,0.0,...,91.0,0.0,0.0,0.0,0.0,65.0,0.0,216.0,0.0,234.0
2022-09-01 05:00:00+02:00,566.0,465.0,0.0,819.0,0.0,97.0,372.0,0.0,143.0,0.0,...,101.0,0.0,0.0,0.0,0.0,402.0,0.0,204.0,0.0,244.0
2022-09-01 06:00:00+02:00,564.0,465.0,0.0,821.0,0.0,161.0,415.0,0.0,306.0,0.0,...,107.0,0.0,0.0,0.0,0.0,406.0,159.0,207.0,0.0,262.0
2022-09-01 07:00:00+02:00,564.0,465.0,0.0,822.0,0.0,219.0,420.0,0.0,330.0,0.0,...,116.0,0.0,0.0,0.0,0.0,406.0,381.0,211.0,0.0,279.0


In [17]:
df_plants_timeseries = pd.DataFrame()

s_index = 0
for p_index, plant in df_plants[0:3].iterrows():
    print(p_index)
    for t_index, time_row in df_gen.iterrows():
        df_plants_timeseries.loc[s_index, 'plant_id'] = p_index
        df_plants_timeseries.loc[s_index, 'timestamp'] = t_index
        df_plants_timeseries.loc[s_index, 'type'] = plant['type']
        df_plants_timeseries.loc[s_index, 'zone'] = plant['zone']
        df_plants_timeseries.loc[s_index, 'installed_capacity'] = plant['installed_capacity']
        
        df_plant_outage = df_outages[(df_outages.start <= t_index) & (df_outages.end >= t_index) & (df_outages.docstatus != 'Cancelled') & (df_outages.production_resource_id == plant['eic'])]
        if len(df_plant_outage) > 0:
            df_plants_timeseries.loc[s_index, 'available_capacity'] = df_plant_outage.iloc[0]['avail_qty']
        else:
            df_plants_timeseries.loc[s_index, 'available_capacity'] = plant['installed_capacity']
        s_index += 1

0
1
2


In [18]:
display(df_plants_timeseries)

,plant_id,timestamp,type,zone,installed_capacity,available_capacity
0,0.0,2022-09-01 00:00:00,natural_gas,AT,838.0,838.0
1,0.0,2022-09-01 01:00:00,natural_gas,AT,838.0,838.0
2,0.0,2022-09-01 02:00:00,natural_gas,AT,838.0,838.0
3,0.0,2022-09-01 03:00:00,natural_gas,AT,838.0,838.0
4,0.0,2022-09-01 04:00:00,natural_gas,AT,838.0,838.0
...,...,...,...,...,...,...
15187,2.0,2023-03-30 19:00:00,natural_gas,AT,216.8,216.8
15188,2.0,2023-03-30 20:00:00,natural_gas,AT,216.8,216.8
15189,2.0,2023-03-30 21:00:00,natural_gas,AT,216.8,216.8
15190,2.0,2023-03-30 22:00:00,natural_gas,AT,216.8,216.8
